# Tensorflow.org

https://www.tensorflow.org/get_started/get_started

In [3]:
# Import dependencies
import tensorflow as tf

In [6]:
# Create a session
sess = tf.Session()

### linear model

In [4]:
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

In [7]:
# tf.Variables are not set until you run init
init = tf.global_variables_initializer()
sess.run(init)

In [9]:
# Test it out
print(sess.run(linear_model, {x:[2,4,6,8]}))

[ 0.30000001  0.90000004  1.5         2.10000014]


In [15]:
# Create a loss function to dertermine close/far the model is from the data
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas) # sum the squares of delta
print(sess.run(loss, {x:[2,4,6,8], y:[0,-1,-2,-3]}))

30.0


Now doing it automatically with tf.train API

In [16]:
# Use gradient descent to optimize the result from our loss function
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [21]:
# Training data (data,labels)
x_train = [2,3,4,5]
y_train = [-1,-2,-3,-4]

# Run train 1000 times aka epochs
for i in range(1000):
    sess.run(train, {x:x_train, y:y_train})
    
print(sess.run([W, b, loss], {x:x_train, y:y_train}))

[array([-1.], dtype=float32), array([ 1.], dtype=float32), 0.0]


### MNIST